In [5]:
""" This time face_recognition using Deep Learning is applied"""

from PIL import Image, ImageDraw
import face_recognition
import os
from pathlib import Path
import cv2

BASE_DIR = Path().resolve()
# images directory
image_dir_1 = os.path.join(BASE_DIR, "images/khai/1.jpg")
image_dir_2 = os.path.join(BASE_DIR, "images/James/1.jpg")
image_dir_3 = os.path.join(BASE_DIR, "images/Kyrie/1.jpg")

# Load the known images
image_of_person_1 = face_recognition.load_image_file(image_dir_1)
image_of_person_2 = face_recognition.load_image_file(image_dir_2)
image_of_person_3 = face_recognition.load_image_file(image_dir_3)

# Get the face encoding of each person. This can fail if no one is found in the photo.
person_1_face_encoding = face_recognition.face_encodings(image_of_person_1)[0]
person_2_face_encoding = face_recognition.face_encodings(image_of_person_2)[0]
person_3_face_encoding = face_recognition.face_encodings(image_of_person_3)[0]

# Create a list of all known face encodings
known_face_encodings = [
    person_1_face_encoding,
    person_2_face_encoding,
    person_3_face_encoding
]

known_face_names = [
    "Khai",
    "James",
    "Kyrie"
]



# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

    
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    reet, frame = cap.read()
    
    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=1, fy=1)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]
        
        # Load the image we want to check
    unknown_image = rgb_small_frame


     # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(unknown_image)
        face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance = 0.6)
            name = "Unknown"

            # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame
    
    
    
    
    
    # Loop through each face found in the unknown image
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 1
        right *= 1
        bottom *= 1
        left *= 1

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (255, 255, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (255, 255, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (0, 0, 255), 1)
        
    # Display the resulting frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# When everything doen, release the capture
cap.release()
cv2.destroyAllWindows()